In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import os
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

sns.set_style("whitegrid")

In [70]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Ronak
[nltk_data]     Desai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [71]:
train_set = pd.read_json('train.json')
train_set.drop(columns=['id'], inplace=True)
display(train_set.sample(5))
print('number of data points: ', len(train_set))

,cuisine,ingredients
27913,french,"[Madeira, chopped fresh thyme, whipping cream,..."
33741,moroccan,"[caraway seeds, fresh ginger root, purple onio..."
31813,filipino,"[cider vinegar, dried oregano, bay leaves, min..."
13016,mexican,"[avocado, water, radishes, cheese, pork should..."
6241,korean,"[ground black pepper, garlic, pears, mirin, ju..."


number of data points:  39774


In [13]:
train_set['tokenized_ingredients'] = train_set['ingredients'].apply(lambda x: ' '.join(x))

In [14]:
train_set.head(5)

,cuisine,ingredients,tokenized_ingredients
0,greek,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olives grape tomatoes ga...
1,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomatoes ground...
2,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggs pepper salt mayonaise cooking oil green c...
3,indian,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt
4,indian,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallots cornflour cayenne pepper...


In [8]:
nltk.pos_tag(nltk.word_tokenize('plain flour ground pepper salt tomatoes'))

[('plain', 'RB'),
 ('flour', 'JJ'),
 ('ground', 'NN'),
 ('pepper', 'IN'),
 ('salt', 'NN'),
 ('tomatoes', 'NNS')]

In [55]:
new_tokenized_ingredients = []
for entry in train_set['tokenized_ingredients']:
    entry = re.sub(r'\([^)]*\)', '', entry)
    entry = re.sub(r'(\d)', '', entry)
    entry = re.sub(u'\w*\u2122', '', entry)
    entry = re.sub(u'\w*\u00AE', '', entry)
    
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(entry)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    entry=' '.join(filtered_sentence)
    new_tokenized_ingredients.append(entry)
train_set['tokenized_ingredients'] = new_tokenized_ingredients
train_set.iloc[1140:1170, :]

,cuisine,ingredients,tokenized_ingredients
1140,southern_us,"[melted butter, flour, sugar, peaches, eggs, v...",melted butter flour sugar peaches eggs vanilla
1141,british,"[pastry dough, fresh parsley, medium eggs, but...",pastry dough fresh parsley medium eggs butter ...
1142,chinese,"[bean threads, soy sauce, ginger, garlic chili...",bean threads soy sauce ginger garlic chili sau...
1143,mexican,"[minced garlic, cumin, tomato sauce, long-grai...",minced garlic cumin tomato sauce long-grain ri...
1144,italian,"[water, ground black pepper, garlic, dried ore...",water ground black pepper garlic dried oregano...
1145,mexican,"[purple onion, kosher salt, tomatoes, basil le...",purple onion kosher salt tomatoes basil leaves
1146,thai,"[pandanus leaf, grated coconut, salt, tapioca ...",pandanus leaf grated coconut salt tapioca flou...
1147,jamaican,"[soy sauce, jalapeno chilies, orange juice, li...",soy sauce jalapeno chilies orange juice lime j...
1148,cajun_creole,"[olive oil, hot sauce, thyme, dried oregano, t...",olive oil hot sauce thyme dried oregano turkey...
1149,vietnamese,"[fish sauce, pork blade steaks, light brown su...",fish sauce pork blade steaks light brown sugar...


In [ ]:
# count_by_cuisine = df.groupby(['cuisine']).count()
# count_by_cuisine['recipe count'] = count_by_cuisine['ingredients']
# cuisines = count_by_cuisine.index.values
# count_by_cuisine.drop(columns=['ingredients'], inplace=True)
# display(count_by_cuisine)

# ingredients_needed = np.zeros(5000)
# for i in range(len(ingredients_needed)):
#     # Ingredients that are present in more than i recipes
#     ingredients_needed[i] = len(ingredient_count_df[ingredient_count_df.sum(axis=1) > i])
# num_recipes_in_1pct = round(len(df)/100)
# most_popular_ingredients_df = ingredient_count_df[ingredient_count_df.sum(axis=1) > num_recipes_in_1pct]
# num_ingredients_in_1pct = len(most_popular_ingredients_df)
# display(most_popular_ingredients_df)
# print(most_popular_ingredients_df.index.values)
# print(num_ingredients_in_1pct, ' ingredients that are present in more than 1% of recipes.')

# ingredients = most_popular_ingredients_df.index.values
# stds = np.zeros(num_ingredients_in_1pct)
# i = 0
# for ingredient in ingredients: 
#     y = most_popular_ingredients_df.loc[[ingredient]].values.reshape(-1) / count_by_cuisine['recipe count'].values
# #     plt.figure()
# #     plt.title('Prevalence of ' + ingredient + ' in different cuisines (normalized by number of recipes)')  
# #     plt.bar(cuisines, y - np.mean(y))
# #     plt.ylabel('deviation from mean')
# #     plt.xticks(rotation='vertical')
# #     plt.ylim([-1,1])
# #     plt.show()
#     stds[i] = np.std(y)
#     i+=1
# n = 100
# best_ings = np.flip(ingredients[np.argsort(stds)][-n:])
# print(n, ' best ingredients from std score: ', best_ings)